## StyleGan image generation

In [ ]:
#%tensorflow_version 1.x
!nvidia-smi
!pip install tensorflow==1.15

Wed Sep  7 13:53:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install numpy==1.19.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
stylegan2_repo_url  = 'https://github.com/dvschultz/stylegan2-ada' # or https://github.com/NVlabs/stylegan2-ada
project_path        = drive_path / 'MyDrive' / 'StyleGAN2-ADA'
stylegan2_repo_path = project_path / 'stylegan2-ada'

# Create project folder if inexistant
if not project_path.is_dir():
    %mkdir "{project_path}"
%cd "{project_path}"

for dir in ['in', 'out', 'datasets', 'training']:
    if not (project_path / dir).is_dir():
        %mkdir {dir}
if not (project_path / 'datasets' / 'source').is_dir():
    %mkdir "{project_path / 'datasets' / 'source'}"

# Download StyleGAN2-ada
!git config --global user.name "ArthurFDLR"
!git config --global user.email "arthfind@gmail.com"
if stylegan2_repo_path.is_dir():
    !git -C "{stylegan2_repo_path}" fetch origin
    !git -C "{stylegan2_repo_path}" checkout origin/main -- *.py
else:
    print("Install StyleGAN2-ADA")
    !git clone {stylegan2_repo_url}

/content/drive/MyDrive/StyleGAN2-ADA


Now, we can generate images. It will take about 5 seconds for 10 images, if you have google colab pro + .In the following kernel you can see the additional commands

In [ ]:
%pip install opensimplex
%pip install imageio==2.4.1
!python "{stylegan2_repo_path / 'generate.py'}" generate-images --help 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
usage: generate.py generate-images [-h] --network NETWORK_PKL --seeds SEEDS
                                   [--trunc TRUNCATION_PSI]
                                   [--class CLASS_IDX] [--create-grid]
                                   [--outdir DIR] [--save_vector] [--fixnoise]
                                   [--jpg_quality JPG_QUALITY]

optional arguments:
  -h, --help            show this help message and exit
  --network NETWORK_PKL
                        Network pickle filename
  --seeds SEEDS         List of random seeds
  --trunc TRUNCATION_PSI
                        Truncation psi (default: 0.5)
  --class CLASS_IDX     Class label (default: unconditional)
  --create-grid         Add flag to save the generated images in a grid
  --outdir DIR          Root directory for run 

In [ ]:
from numpy import random
seed_init = random.randint(10000)
nbr_images = 10           #number of images generated in this batch

generation_from = 'pickle_path'   #put path of the model here
out_dir = "outdir"           #put the directory of the output 
!python "{stylegan2_repo_path / 'generate.py'}" generate-images \
    --outdir=out_dir --trunc=-0.5 \                                                  
    --seeds={seed_init}-{seed_init+nbr_images-1} \
    --network={generation_from}

Loading networks from "/content/drive/MyDrive/StyleGAN2-ADA/training/special64/00009-tfr-auto1-bgc-resumecustom/network-snapshot-000080.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Compiling... Loading... Done.
Generating image for seed 3044 (0/10) ...
Generating image for seed 3045 (1/10) ...
Generating image for seed 3046 (2/10) ...
Generating image for seed 3047 (3/10) ...
Generating image for seed 3048 (4/10) ...
Generating image for seed 3049 (5/10) ...
Generating image for seed 3050 (6/10) ...
Generating image for seed 3051 (7/10) ...
Generating image for seed 3052 (8/10) ...
Generating image for seed 3053 (9/10) ...


# Image feature vector 
this requires a tf2 environment. It is recommended to simply disconnect and reconnect or download the newest tf2 package via pip. 

In [ ]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import csv
import matplotlib.pyplot as plt

In [ ]:
data_dir = r"Ground Truth Dataset path"
image_dir = r"generated images path"

In [ ]:
import tensorflow as tf
def totensor(path):
# load image via tf.io
  img = tf.io.read_file(path)

  # convert to tensor (specify 3 channels explicitly since png files contains additional alpha channel)
  # set the dtypes to align with pytorch for comparison since it will use uint8 by default
  tensor = tf.io.decode_jpeg(img) #, channels=3, dtype=tf.dtypes.float32
  # (384, 470, 3)

  # resize tensor to 224 x 224
  tensor = tf.image.resize(tensor, [224, 224])
  # (224, 224, 3)
  tensor1 = tensor[None,:,:,:]
  # add another dimension at the front to get NHWC shape
  input_tensor = tf.expand_dims(tensor, axis=0)
  # (1, 224, 224, 3)
  return input_tensor

In [ ]:
module = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/5",
                   trainable=False),  # Can be True, see below.
    #tf.keras.layers.Dense(16, activation='softmax')
])
module.build([None, 224, 224, 3])
module.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1792)              4363712   
                                                                 
Total params: 4,363,712
Trainable params: 0
Non-trainable params: 4,363,712
_________________________________________________________________


In [ ]:
os.mkdir("/content/projector/")
with open('/content/projector/feature_vecs.tsv', 'w') as fw:
    csv_writer = csv.writer(fw, delimiter='\t')
    for image_path in os.listdir(data_dir):

      inp = totensor(os.path.join(data_dir,image_path))
      fvecs = module.predict(inp,steps=1)
      csv_writer.writerows(fvecs)
    for i in os.listdir(image_dir):
      csv_writer.writerows(module.predict(totensor(os.path.join(image_dir,i)),steps=1))

In [ ]:
with open('/content/projector/metadata.tsv', 'w') as fw:    
    csv_writer = csv.writer(fw, delimiter='\t')
    for image_path in os.listdir(data_dir):
      csv_writer.writerows("o")
    for i in os.listdir(image_path):
      csv_writer.writerow([str(i)])

Afterwards, download the projector directory and go to https://projector.tensorflow.org/. There, click the button 'Load' and select the files in the projector directory with corresponding names.

# post-process images
This can be done with regular methods or AI models, but for simplicity, normal methods will suffice for now. As for more complicated methods, we'll use https://tfhub.dev/captain-pool/esrgan-tf2/1 for super resolution and use the api of e.g. photoshop or coreldraw for noise removal.

In [ ]:
import os
from PIL import Image

In [ ]:
PATH = "generated_images_dir"
results_path = "resulting directories path"
for q in os.listdir(PATH):
  im = Image.open(os.path.join(PATH,q))
  d = 0
  pathseed = os.path.join(results_path,q)
  if not os.path.isdir(pathseed):

    os.mkdir(os.path.join(results_path,q))
  for i in range(16):
      x = i*64
      w = i*64+64
      for j in range(16):
          y = j*64
          h = j*64 + 64
          im1 = im.crop((x, y,w, h))
          im1 = im1.save(results_path + "/{}/{}.jpg".format(q,d))
          d += 1

fontforge script, which is supposed to be run in the python based open source program fontforge. For it to work, one needs to fill in all the path variables, create a new empty font and run the script via the execute script button in the files tab

In [ ]:
import fontforge                                 #Load the module
import os


uni = "u7684,u4e00,u662f,u4e0d,u4eba,u6709,u4e86,u5728,u4f60,u6211,u4e2a,u5927,u4e2d,u8981,u8fd9,u4e3a,u4e0a,u751f,u65f6,u4f1a,u4ee5,u5c31,u5b50,u5230,u6765,u53ef,u80fd,u548c,u81ea,u4eec,u5e74,u591a,u53d1,u5fc3,u597d,u7528,u5bb6,u51fa,u5173,u957f,u4ed6,u6210,u5929,u5bf9,u4e5f,u5c0f,u540e,u4e0b,u5b66,u90fd,u70b9,u56fd,u8fc7,u5730,u884c,u4fe1,u65b9,u5f97,u6700,u8bf4,u4e8c,u4e1a,u5206,u4f5c,u5982,u770b,u5973,u4e8e,u9762,u6ce8,u522b,u7ecf,u52a8,u516c,u5f00,u73b0,u800c,u7f8e,u4e48,u8fd8,u4e8b,u5df1,u7406,u7ef4,u6ca1,u4e4b,u60c5,u9ad8,u6cd5,u5168,u5f88,u65e5,u4f53,u91cc,u5de5,u5fae,u8005,u5b9e,u529b,u505a,u7b49,u6c34,u52a0,u5b9a,u679c,u53bb,u6240,u65b0,u6d3b,u7740,u8ba9,u8d77,u5e02,u8eab,u95f4,u7801,u54c1,u8fdb,u5b69,u524d,u60f3,u9053,u79cd,u8bc6,u6309,u540c,u8f66,u672c,u7136,u6708,u673a,u6027,u4e0e,u90a3,u65e0,u624b,u7231,u6837,u56e0,u8001,u5185,u90e8,u6bcf,u66f4,u610f,u53f7,u7535,u5176,u91cd,u5316,u5f53,u53ea,u6587,u5165,u4ea7,u5408,u4e9b,u5979,u4e09,u8d39,u901a,u4f46,u611f,u5e38,u660e,u7ed9,u4e3b,u540d,u4fdd,u63d0,u5c06,u5143,u8bdd,u6c14,u4ece,u6559,u76f8,u5e73,u7269,u573a,u91cf,u8d44,u77e5,u6216,u5916,u5ea6,u91d1,u6b63,u6b21,u671f,u95ee,u653e,u5934,u4f4d,u5b89,u6bd4,u771f,u52a1,u7537,u7b2c,u89e3,u539f,u5236,u533a,u6d88,u8def,u53ca,u8272,u7f51,u82b1,u628a,u6253,u5403,u7cfb,u56de,u6b64,u5e94,u53cb,u9009,u4ec0,u8868,u5546,u518d,u4e07,u5988,u88ab,u5e76,u4e24,u9898,u670d,u5c11,u98ce,u98df,u53d8,u5bb9,u5458,u4ea4,u513f,u8d28,u5efa,u6c11,u4ef7,u517b,u623f,u95e8,u9700,u5f71,u8bf7,u5229,u7ba1,u767d,u7b80,u53f8,u4ee3,u53e3,u53d7,u56fe,u5904,u624d,u7279,u62a5,u57ce,u5355,u897f,u5b8c,u4f7f,u5df2,u76ee,u6536,u5341,u5019,u5c71,u6570,u5c55,u5feb,u5f3a,u5f0f,u7cbe,u7ed3,u4e1c,u5e08,u6c42,u63a5,u81f3,u6d77,u7247,u6e05,u5404,u76f4,u5e26,u7a0b,u4e16,u5411,u5148,u4efb,u8bb0,u6301,u683c,u603b,u8fd0,u8054,u8ba1,u89c9,u4f55,u592a,u7ebf,u53c8,u514d,u70ed,u4ef6,u6743,u8c03,u4e13,u533b,u4e50,u6548,u795e,u51fb,u8bbe,u94b1,u5065,u6d41,u7531,u89c1,u53f0,u51e0,u589e,u75c5,u6295,u6613,u5357,u5bfc,u529f,u4ecb,u8bc1,u8d70,u4eca,u5149,u670b,u5373,u89c6,u9020,u60a8,u7acb,u6539,u6bcd,u63a8,u773c,u590d,u653f,u4e70,u4f20,u8ba4,u975e,u57fa,u5b9d,u8425,u9662,u56db,u4e60,u8d8a,u5305,u6e38,u8f6c,u6280,u6761,u606f,u8840,u79d1,u96be,u89c4,u4f17,u559c,u4fbf,u521b,u5e72,u754c,u793a,u5e7f,u7ea2,u4f4f,u6b22,u6e90,u6307,u8be5,u89c2,u8bfb,u4eab,u6df1,u6cb9,u8fbe,u544a,u5177,u53d6,u8f7b,u5eb7,u578b,u5468,u88c5,u5f20,u4e94,u6ee1,u5e97,u4eb2,u6807,u67e5,u80b2,u914d,u5b57,u7c7b,u4f18,u59cb,u6574,u636e,u8003,u6848,u5317,u5b83,u5ba2,u706b,u5fc5,u8d2d,u529e,u793e,u547d,u5473,u6b65,u62a4,u672f,u9605,u5427,u7d20,u6237,u5f80,u83dc,u9002,u8fb9,u5374,u5931,u8282,u6599,u8f83,u5f62,u8fd1,u7ea7,u51c6,u76ae,u8863,u4e66,u9a6c,u8d85,u7167,u503c,u7236,u600e,u8bd5,u7a7a,u5207,u627e,u534e,u4f9b,u7c73,u4f01,u52a9,u53cd,u671b,u9999,u8db3,u798f,u4e14,u6392,u9633,u7edf,u672a,u6cbb,u51b3,u786e,u9879,u9664,u4f4e,u6839,u5c81,u5219,u767e,u5907,u50cf,u65e9,u9886,u9152,u6b3e,u9632,u96c6,u73af,u5bcc,u8d22,u8ddf,u81f4,u7626,u901f,u62e9,u6e29,u9500,u56e2,u79bb,u5462,u8bae,u8bba,u5417,u738b,u5dde,u6001,u601d,u53c2,u8bb8,u8fdc,u8d23,u5e03,u7f16,u968f,u7ec6,u6625,u514b,u542c,u51cf,u8a00,u62db,u7ec4,u666f,u7a7f,u9ec4,u836f,u8089,u552e,u80a1,u9996,u9650,u68c0,u4fee,u9a8c,u5171,u7ea6,u6bb5,u7b11,u6d17,u51b5,u7eed,u5e95,u56ed,u5e2e,u5f15,u5a5a,u4efd,u5386,u6d4e,u9669,u58eb,u9519,u8bed,u6751,u4f24,u5c40,u7968,u5584,u6821,u6218,u9645,u76ca,u804c,u591f,u665a,u6781,u652f,u5b58,u65c5,u6545,u542b,u7b97,u9001,u8bc9,u7559,u89d2,u677e,u79ef,u7701,u4ec5,u6c5f,u5883,u79f0,u534a,u661f,u5347,u8c61,u6750,u9884,u7fa4,u83b7,u9752,u7ec8,u5bb3,u80a4,u5c5e,u663e,u5361,u9910,u94f6,u58f0,u7ad9,u961f,u843d,u5047,u53bf,u996d,u8865,u7814,u8fde,u5fb7,u54ea,u949f,u9047,u9ed1,u53cc,u5f85,u6bd2,u65ad,u5145,u667a,u6f14,u8bb2,u538b,u519c,u613f,u5c3d,u62c9,u7c89,u54cd,u6b7b,u724c,u53e4,u8d27,u73a9,u82e6,u7387,u5343,u65bd,u86cb,u5668,u697c,u75db,u7a76,u7761,u72b6,u8ba2,u4e49,u7edd,u77f3,u4eae,u52bf,u97f3,u642d,u59d4,u65af,u5c45,u674e,u7d27,u575a,u8138,u72ec,u4f9d,u4e3d,u4e25,u6b62,u7597,u53f3,u559d,u9e21,u725b,u6797,u677f,u67d0,u8d1f,u4eac,u4e30,u53e5,u8bc4,u878d,u519b,u61c2,u5438,u5212,u5ff5,u592b,u5c42,u964d,u54e6,u7a0e,u8c46,u5f69,u5b98,u7edc,u80f8,u62ff,u753b,u5c14,u9f99,u5bdf,u73ed,u6784,u79d8,u5426,u53eb,u7403,u5e78,u5ea7,u6162,u5174,u4f5b,u5ba4,u554a,u5747,u4ed8,u6a21,u534f,u4e92,u7f6e,u822c,u82f1,u51c0,u6362,u77ed,u5de6,u7248,u8bfe,u8336,u7b56,u6bdb,u505c,u6cb3,u80a5,u7b54,u826f,u4e45,u627f,u63a7,u6fc0,u8303,u7ae0,u4e91,u666e,u5957,u53e6,u5956,u987b,u4f8b,u5199,u7075,u62c5,u5fd7,u987e,u8349,u9547,u9000,u5e0c,u8c22,u7238,u91c7,u516d,u9c7c,u56f4,u5bc6,u5ead,u8111,u5947,u516b,u5356,u7ae5,u571f,u5708,u8c01,u62e5,u7cd6,u76d1,u751a,u6015,u8d35,u987a,u9c9c,u51b7,u5dee,u68a6,u8b66,u62cd,u94c1,u4ebf,u4e89,u591c,u80cc,u6c38,u8857,u5f8b,u996e,u7ee7,u523b,u521d,u7a81,u5012,u8058,u6728,u719f,u5a46,u5217,u9891,u867d,u521a,u5986,u4e3e,u5c1a,u6c7d,u66fe,u811a,u5976,u7834,u9759,u9a7e,u5757,u84dd,u9178,u6838,u9505,u827a,u7eff,u535a,u989d,u9648,u5750,u9760,u5de7,u6389,u98de,u76d8,u5e01,u817f,u5df4,u57f9,u82e5,u95fb,u53f2,u4e9a,u7eb8,u75c7,u5b63,u53f6,u4e61,u4e1d,u8be2,u5267,u793c,u4e03,u5740,u6dfb,u7ec7,u7565,u865a,u8fce,u6444,u4f59,u4e4e,u7f3a,u80c3,u7206,u57df,u59bb,u7ec3,u8350,u4e34,u4f73,u5e9c,u8ffd,u60a3,u6811,u989c,u8bda,u4f34,u6e56,u8d34,u5348,u56f0,u4f3c,u6d4b,u809d,u5f52,u5b81,u6696,u7eb3,u5b9c,u963f,u5f02,u536b,u5f55,u6db2,u79c1,u8c08,u6ce1,u60ca,u7d22,u76d0,u6f02,u635f,u7a33,u4f11,u6298,u8baf,u5802,u6000,u60e0,u6c64,u7eaa,u6563,u85cf,u6e7f,u900f,u4ee4,u51b0,u5987,u9ebb,u9192,u5ba3,u6297,u5178,u6267,u79c0,u808c,u8bad,u5218,u6025,u8d76,u64ad,u82cf,u6de1,u9769,u9634,u6279,u76d6,u8170,u80a0,u8131,u5370,u786c,u4fc3,u51b2,u5e8a,u52aa,u810f,u8dd1,u96c5,u5385,u7f57,u60ef,u65cf,u59d0,u72af"
unicodes = uni.split(",")

PATH = r"PATH"   #put the path of all the images here
for h in os.listdir(PATH):
  
  d= fontforge.open(r"font.ttf")                  #put the path of a existing example font here, so we can get the english alphabet which we did not generate
  for i in range(len(os.listdir(os.path.join(PATH,h)))-1):
    
    glyph = d.createMappedChar(unicodes[i])
    glyph.clear()  
    glyph.importOutlines(PATH + "\{}\{}.jpg".format(h,i))
    glyph.autoTrace()
  d.generate(result_path + "{}.ttf".format(h))          #put the path of the results here
  d.close()
        

In [ ]:
if enc in n:
      glyph = n[enc]
    else: